# Sample: F1 vs Accuracy Bubble Chart

This notebook demonstrates the **Tier 1 → Tier 2 → Tier 3 promotion workflow**
by recreating the dashboard's "F1 vs Accuracy" bubble chart.

## Production original

The production version lives in `reports/dashboard.qmd` (lines 261–309).
It uses Mosaic/vgplot + DuckDB-WASM in the browser. Here we recreate it
with **pyobsplot + DuckDB Python** for rapid inline iteration.

## Workflow

```
1. [THIS NOTEBOOK]  Shape query + plot spec with pyobsplot
2. [playground.qmd]  Paste Observable Plot spec, verify in Quarto preview
3. [dashboard.qmd]   Promote to production (translate to Mosaic if needed)
```

---
## Step 1: Connect to data

Load the same Parquet files the dashboard uses. This ensures our prototype
queries work identically in production.

In [ ]:
import duckdb
import json
from pathlib import Path

DATA_DIR = Path("../reports/data")

db = duckdb.connect()
db.execute(f"CREATE TABLE metrics AS SELECT * FROM '{DATA_DIR / 'metrics.parquet'}'")
db.execute(f"CREATE TABLE runs AS SELECT * FROM '{DATA_DIR / 'runs.parquet'}'")

# Load model sizes JSON (same file the dashboard uses for bubble radius)
model_sizes = json.loads((DATA_DIR / "model_sizes.json").read_text())
sizes = {d["model_type"]: d["param_count_M"] for d in model_sizes["data"]}

print(f"metrics: {db.sql('SELECT COUNT(*) FROM metrics').fetchone()[0]} rows")
print(f"runs: {db.sql('SELECT COUNT(*) FROM runs').fetchone()[0]} rows")
print(f"model sizes: {sizes}")

---
## Step 2: Shape the query

The dashboard query extracts dataset from `run_id` via `split_part()`,
then joins with model sizes for the bubble radius. We do the same here
and add `param_count_M` as a Python column.

In [ ]:
df = db.sql("""
    SELECT
        split_part(run_id, '/', 1) AS dataset,
        model,
        f1,
        accuracy,
        auc
    FROM metrics
    WHERE f1 IS NOT NULL AND accuracy IS NOT NULL
""").df()

# Add parameter count (mirrors the JS: sizes.get(d.model) || 0.1)
df["param_count_M"] = df["model"].map(lambda m: sizes.get(m, 0.1))

print(f"{len(df)} data points")
df.head(10)

---
## Step 3: Build the Observable Plot spec

This is the key transfer artifact. The `Plot.plot({...})` call below
uses the **exact same Observable Plot API** as the web playground and
production dashboard. When this looks right, copy the spec dict directly.

### Design choices (matching production)
- **Color**: `gat=#3fb950`, `vgae=#58a6ff`, `fusion=#bc8cff` (GitHub dark palette)
- **Size**: bubble radius encodes parameter count (millions)
- **Reference line**: y=x dashed line (F1 = Accuracy)
- **Domain**: [0.4, 1.01] on both axes to focus on the interesting range

In [ ]:
from pyobsplot import Plot

Plot.plot({
    "width": 680,
    "height": 400,
    "color": {
        "legend": True,
        "domain": ["gat", "vgae", "fusion"],
        "range": ["#3fb950", "#58a6ff", "#bc8cff"],
    },
    "r": {"range": [4, 30]},
    "x": {"label": "F1 Score", "domain": [0.4, 1.01]},
    "y": {"label": "Accuracy", "domain": [0.4, 1.01]},
    "marks": [
        Plot.dot(df, {
            "x": "f1",
            "y": "accuracy",
            "r": "param_count_M",
            "fill": "model",
            "fillOpacity": 0.6,
            "stroke": "model",
            "tip": True,
        }),
        # y=x reference line
        Plot.line(
            [{"x": 0.4, "y": 0.4}, {"x": 1.0, "y": 1.0}],
            {"x": "x", "y": "y", "stroke": "#8b949e", "strokeDasharray": "4,4", "strokeWidth": 1},
        ),
    ],
})

---
## Step 4: Iterate on variations

The notebook lets you quickly try alternative encodings without waiting
for Quarto to rebuild. For example, faceting by dataset:

In [ ]:
Plot.plot({
    "width": 680,
    "height": 300,
    "color": {
        "legend": True,
        "domain": ["gat", "vgae", "fusion"],
        "range": ["#3fb950", "#58a6ff", "#bc8cff"],
    },
    "facet": {"data": df, "x": "dataset"},
    "x": {"label": "F1 Score"},
    "y": {"label": "Accuracy"},
    "marks": [
        Plot.dot(df, {
            "x": "f1",
            "y": "accuracy",
            "fill": "model",
            "r": 6,
            "tip": True,
        }),
        Plot.frame(),
    ],
})

---
## Step 5: Promotion notes

### → `reports/playground.qmd` (Tier 2)

Copy the Plot spec from Step 3 into a scratch cell. The **only translation needed**
is Python → JS syntax:

| Python (pyobsplot) | JavaScript (OJS cell) |
|---|---|
| `True` / `False` | `true` / `false` |
| `Plot.plot({...})` | `Plot.plot({...})` (identical) |
| `df` (pandas DataFrame) | `data` (JS array from SQL) |
| `"tip": True` | `tip: true` |

### → `reports/dashboard.qmd` (Tier 3)

The production dashboard uses **Mosaic/vgplot** (not raw Observable Plot)
for cross-filtering. Translation from Observable Plot → Mosaic:

| Observable Plot | Mosaic/vgplot |
|---|---|
| `Plot.dot(data, {x, y})` | `vg.dot(vg.from("metrics"), {x, y})` |
| `Plot.plot({marks: [...], width: 680})` | `vg.plot(mark, vg.width(680))` |
| `{color: {legend: true}}` | `vg.colorLegend(true)` |
| Data is a JS array | Data is a DuckDB table name |

The SQL query (`split_part`, `WHERE`) copies verbatim — DuckDB Python
and DuckDB-WASM use the same SQL dialect.